# Determining True Rating of Yelp Restaurants

Through data science analysis we aim to provide insight about the "true" rating of a restaraunt on Yelp. We define the "true" rating to be the rating that a restaurant will converge to over time based upon various features of the restaruant we have scraped from Yelp. This will be particularly useful for restuarants with few ratings that are curious as to what their true rating will be over time given features about their restaurant. Features used within our model include TODO

## Data Collection

We collected the dataset we plan on analyzing with the use of the Yelp API and Beautiful Soup. We also used Pickle to serialize our dataset so we don't have to continously query the Yelp API to get our data set. 

In [5]:
# setup library imports
import io, time, json
import requests
from bs4 import BeautifulSoup
import cPickle as pickle
import numpy as np 
import matplotlib.pyplot as plt

# import yelp client library
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator


RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

ImportError: numpy.core.multiarray failed to import

In [18]:
def retrieve_html(url):
    html = requests.get(url)
    return (html.status_code, html.content)

""" Gets restaurant features from yelp html given a url for a restaurants page
    returns a dictionary of the available features on the page (keys) with their 
    corresponding values """
def scrapeYelpFeatures(url):
    (yolo, html) = retrieve_html(url)
    
    try:
        imgs =  re.search(r"See all [0-9]+", html).group()
        imgs = [int(s) for s in imgs.split() if s.isdigit()][0]
    except:
        imgs = 0
    root = BeautifulSoup(html, "html.parser")
    s = root.findAll("div", {'class' : 'ywidget'}) #, { "class" : "review review--with-sidebar" })
    #s = root.find("h3", {'name': 'More business info'})
    
    biz_features = {"number_of_images" : imgs}
    for i in s:
        h3_elem = i.find("h3")
        if h3_elem != None:
            if 'More business info' in h3_elem:
                biz_attr = i.find('ul', {'class' : 'ylist'}).findAll('dt', {'class' : 'attribute-key'})
                biz_val = i.find('ul', {'class' : 'ylist'}).findAll('dd')

                for (l,r) in zip(biz_attr, biz_val):
                    biz_features[l.next_element.strip()] = r.next_element.strip()
    return biz_features

224

In [4]:
def authenticate(config_filepath):
    creds = json.load(open(config_filepath))
    auth = Oauth1Authenticator(**creds)
    return Client(auth)

In [5]:
def yelp_search(client, query):
    """
    Make an authenticated request to the Yelp API.

    Args:
        query (string): Search term

    Returns:
        total (integer): total number of businesses on Yelp corresponding to the query
        businesses (list): list of yelp.obj.business.Business objects
    """
    res = client.search(query)
    return (res.total, res.businesses)

In [4]:
def get_cities_info(client, food, cities):
    d = {}
    for city in cities:
        print "requesting: ", city,
        d["%s_%s"%(city,food)] = all_restaurants(client,city,food)
        print ": done"
    #d = {("%s_%s"%(city,food)):all_restaurants(client,city,food) for city in cities}
    valids = filter(lambda x: len(d[x]) >= 10, d)
    res = {v:d[v] for v in valids}
    return res

def get_all_cities():
    with open("cities.txt",'r') as f:
        lines = map(lambda l: ", ".join(l.rstrip().split("\t")[1:3]),f.readlines())
        return lines
    
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
def save_pickle(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

## Data Processing

In [9]:
seafood_data = load_pickle("cities_seafood.pkl")
review_data = get_all_city_data(seafood_data)

In [ ]:

x = map(lambda z: review_data[z]["num_restaurants"], review_data)
y = map(lambda z:   review_data[z]["avg_review"], review_data)
fit = np.polyfit(x,y,1)
print fit
fit_fn = np.poly1d(fit) 
plt.plot(x,y, 'yo', x, fit_fn(x), '--k')
plt.ylabel('ratings')
plt.xlabel("number of seafood restaurants in city")
plt.show()

[  2.14136645e-04   3.57854820e+00]


In [7]:
sorted_cities = sorted(list(review_data), key = lambda z: -review_data[z]["avg_review"])
ratings = []
for city in sorted_cities:
    ratings.append([city, review_data[city]["avg_review"], review_data[city]["num_restaurants"]])

## Data Exploration and Visualization

Our primary concerns when performing our analysis was to ensure we could normalize the data for biases. In particular the population within the United States is very different demographically between cities, which adds to the noise in ratings given by each of these demographics. We were primarily concerned about the relative expectations of different cities due to these differences in demographics (populations’ sentiment, density, and socioeconomics). For example, a city that is not known for its seafood might make people lower their expectations when going to a seafood restaurant and therefore they give a higher rating than they would if they were in a city that is known for its seafood. To circumvent this issue we plan to explore a correlation between the average ratings given to seafood restaurants of a certain region and the number of such restaurants within the region. We plan to use this analysis to normalize ratings across cities to best assess the true ratings of restaurants unhindered by the biases present in separate geographies.

In [2]:
import ggplot

RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

ImportError: numpy.core.multiarray failed to import

## Data Analysis

We will be focusing on determining an adaquate normalization technique.

## Results and Conclusions

We will be using the map library Folium to visualize our results and make some final conclusions.

In [10]:
import folium
from geopy.geocoders import Nominatim
geolocator = Nominatim()
ratings = ratings[1:]
for i in xrange(len(ratings)):
    loc = geolocator.geocode(ratings[i][0])
    ratings[i].append((loc.latitude, loc.longitude))

ImportError: No module named folium

In [9]:
USMap = folium.Map(location=[39.8282, -98.5795], zoom_start = 4)
for rating in ratings:
    locationName = str(rating[0])
    ratingNum = rating[1]
    numRestaurants = rating[2]
    latlong = rating[3]
    if "–" not in locationName:
        folium.CircleMarker(location=[latlong[0], latlong[1]], radius=ratingNum * numRestaurants * 20,
                        popup=locationName, color='#3186cc', fill_color='#3186cc').add_to(USMap)
USMap.save("seafood.html")

In [64]:
%%HTML

In [5]:
test = load_pickle("data.pkl")

In [6]:
type(test)

list

In [7]:
type(test[0])

dict

In [8]:
print test[0]

{u'Accepts Apple Pay': u'No', 'rating': 4.5, u'Accepts Bitcoin': None, u'Drive-Thru': None, u'Alcohol': u'No', u'Noise Level': u'Average', u'Takes Reservations': u'Yes', u'Has TV': u'Yes', u'Attire': u'Casual', u'Ambience': u'Casual', u'Good for Kids': u'Yes', u'Good for Groups': u'Yes', u'Wheelchair Accessible': u'Yes', 'review_count': 158, u'Caters': u'Yes', u'Delivery': u'No', u'Dogs Allowed': u'Yes', u'Best Nights': None, 'location': <yelp.obj.location.Location object at 0x104a8bc90>, u'Coat Check': None, u'Smoking': None, u'Accepts Credit Cards': u'Yes', u'Take-out': u'Yes', u'Ages Allowed': None, u'Offers Military Discount': None, u'Has Pool Table': None, u'By Appointment Only': None, u'Happy Hour': None, u'Outdoor Seating': u'Yes', u'Good for Working': None, u'Waiter Service': u'Yes', u'Wi-Fi': u'No', 'categories': [Category(name=u'Asian Fusion', alias=u'asianfusion'), Category(name=u'Japanese', alias=u'japanese'), Category(name=u'Mexican', alias=u'mexican')], 'name': u'KooKoo G